In [1]:
import cv2
import numpy as np

In [10]:
# Load YOLO model
yolo_config = r"C:\\Users\\DoctorComputers\\Desktop\\Object_Detection\\yolov3.cfg"
yolo_weights = r"C:\\Users\\DoctorComputers\\Desktop\\Object_Detection\\yolov3.weights"
yolo_classes = r"C:\\Users\\DoctorComputers\\Desktop\\Object_Detection\\coco.names"

In [11]:
# Load class labels
with open(yolo_classes, "r") as f:
    classes = [line.strip() for line in f.readlines()]

<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'C:\\\\Users\\\\DoctorComputers\\\\Desktop\\\\Object_Detection\\\\coco.names'

In [ ]:
# Load YOLO model
net = cv2.dnn.readNet(yolo_weights, yolo_config)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    height, width, channels = frame.shape

    # Detect objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process detected objects
    class_ids, confidences, boxes = [], [], []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x, center_y, w, h = (detection[0:4] * [width, height, width, height]).astype("int")
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-maximum suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    # Draw bounding boxes
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
        color = colors[class_ids[i]]
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("YOLO Object Detection", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [13]:
import os

path = r"C:\Users\DoctorComputers\Desktop\Object_Detection"
print(os.listdir(path))  # List all files in the directory

<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'C:\\Users\\DoctorComputers\\Desktop\\Object_Detection'